In [9]:
import pandas as pd 
import numpy as np
import json
from pandas_profiling import ProfileReport
from pandas.api.types import is_numeric_dtype

url = 'https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv'
income_data = pd.read_csv(url)

In [2]:
income_data.head(10)

,x,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,5,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,6,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,7,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,8,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,9,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,10,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [3]:
"""
결측값 파악 및 정의

['?', 'NA', 'na', 'null', 'Null', 'NULL', ' '] 해당 문자들을 결측값으로 인식하며, 결측값 처리여부를 묻는다.
"""
def na_check(df, na_list = ['?', 'NA', 'na', 'null', 'Null', 'NULL', ' ']):

    candidate = [na for na in na_list if na in df.values]

    for c in candidate:
        answer = input(f"{c}를 결측값으로 처리할까요? y 또는 n을 입력하세요")

        if answer is 'y':
            # 결측값은 numpy의 nan을 활용한다.
            df = df.replace({c: np.nan})        
        else:
            print('y또는 n값만 허용합니다')
            break
    
    return(df)

In [4]:
"""
데이터 품질 확인

"""
def dqc(df):
    # 결과를 dict로 저장한다.
    output = dict()
    
    rows = df.shape[0] 
    cols = df.shape[1]
    
    # data 내 전체 결측값을 확인한다.
    total_null = sum(df.isnull().sum())
    
    # 중복되는 row가 있는지 확인한다. 
    duplicate_row = sum(df.duplicated())
    
    # 중복 row의 index를 확인한다.
    duplicate_index = [idx for idx, result in df.duplicated().to_dict().items() if result is True]

    output['dataset'] = {
        'rows': rows,
        'cols': cols,
        'null': total_null,
        'duplicate row': duplicate_row,
        'duplicate row index': duplicate_index
    }

    column_names = df.columns.to_list()
    
    # 깂 전체가 결측값인 column을 확인한다.
    null_count_dict = df.isnull().sum().to_dict()
    all_null_column = [column for column, value in null_count_dict.items() if value == rows]
    
    # 값 전체가 동일한 column을 확인한다.
    all_same_column_dict = np.var(df).to_dict()
    all_same_column = [column for column, var in all_same_column_dict.items() if var == 0]
    
    # 모든값이 중복되는 column을 확인한다.
    duplicate_column_dict = df.transpose().duplicated().to_dict()
    duplicate_column = [column for column, result in duplicate_column_dict.items() if result is True]

    variables = dict()
    string_variable = dict()
    numeric_variable = dict()

    for column_name in column_names:

        temp_dict = dict()
        
        # 값 전체가 결측값인 column은 all_null 값이 1로 입력된다. 
        if column_name in all_null_column:
            temp_dict['all_null'] = 1 
        else: 
            temp_dict['all_null'] = 0

        # column 내에 결측값 갯수를 확인한다.
        temp_dict['null_count'] = null_count_dict[column_name] 
        
        # column 내에 unique 값 갯수를 확인한다.
        unique_value = list(df[column_name].unique())
        unique_count = len(unique_value)
        temp_dict['unique_count'] = unique_count
        
        # 값 전체가 동일한 column은 all_same 값이 1로 입력된다.
        if unique_count == 1:
            temp_dict['all_same'] = 1 
        else: 
            temp_dict['all_same'] = 0
        
        # 중복되는 column일 경우 duplicate 값이 1로 입력된다.
        if column_name in duplicate_column:
            temp_dict['duplicate'] = 1 
        else: 
            temp_dict['duplicate'] = 0

        # is_numeric_dtype 함수를 가지고 column의 string, numeric type을 구분한다.     
        if is_numeric_dtype(df[column_name]) is True:
            temp_dict['min'] = min(df[column_name])
            temp_dict['avg'] = round(sum(df[column_name]) / len(df[column_name]), 2)
            temp_dict['max'] = max(df[column_name])

            numeric_variable[column_name] = temp_dict
        else: 
            values = df[column_name].astype(str)
            values_percent = round(values.groupby(values).count() / len(values), 2)
            temp_dict['values_percent'] = values_percent.to_dict()

            string_variable[column_name] = temp_dict

    output['variable'] = {
        'numeric_variable': numeric_variable,  
        'string_variable': string_variable      
    }

    return(output)

In [5]:
na_check_df = na_check(income_data)

?를 결측값으로 처리할까요? y 또는 n을 입력하세요 y


In [10]:
# 결과를 json으로 저장한다.
json.dump(output, open("output.json", 'w'))